# A solution for NSUCRYPTO 2024

- __Problem 8:__ Unsecure SP-network

Firstly, let's generate all 2x2 S-boxes:

In [2]:
import itertools

S_boxes = []

for S_box in itertools.permutations(range(4), 4):
    print(S_box)
    S_boxes.append(S_box)

(0, 1, 2, 3)
(0, 1, 3, 2)
(0, 2, 1, 3)
(0, 2, 3, 1)
(0, 3, 1, 2)
(0, 3, 2, 1)
(1, 0, 2, 3)
(1, 0, 3, 2)
(1, 2, 0, 3)
(1, 2, 3, 0)
(1, 3, 0, 2)
(1, 3, 2, 0)
(2, 0, 1, 3)
(2, 0, 3, 1)
(2, 1, 0, 3)
(2, 1, 3, 0)
(2, 3, 0, 1)
(2, 3, 1, 0)
(3, 0, 1, 2)
(3, 0, 2, 1)
(3, 1, 0, 2)
(3, 1, 2, 0)
(3, 2, 0, 1)
(3, 2, 1, 0)


A S-Box is linear if it satisfies the property: $$SBox_{i \oplus j \oplus 0} = SBox_i \oplus SBox_j \oplus SBox_0$$ for all $i, j \in \{0, 1,...,|SBox|\}$. We can use this property to check a S-Box is linear or not.

In [4]:
def check_linearity(S_box):
    for i in range(len(S_box)):
        for j in range(len(S_box)):
            if S_box[i ^ j ^ 0] != S_box[i] ^ S_box[j] ^ S_box[0]:
                return False
    return True

Now, we can check all 2x2 S-boxes we are generated above:

In [5]:
for S_box in S_boxes:
    print(check_linearity(S_box))

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


So all 2x2 S-boxes are affine, which means that each S-box can be modeled as the affine function $SBox(x) = A*x + B$, where $A$ is 2x2-matrix in $GF(2)$, $B, x \in \mathbb{F}_2^2$. Therefore, the substitution step can be represented as: $$S(x) = C*x + D = \begin{bmatrix} & & & & A \\ & & & A & \\ & & \dots & & \\ & A & & & \\ A & & & & \end{bmatrix} * x + \begin{bmatrix} B \\ B \\ \vdots \\ B \\ B \end{bmatrix}$$ where $x, D \in \mathbb{F}_2^{32}$ and $C$ is 32x32-matrix in $GF(2)$.

Because $P$ is an arbitrary linear transformation, so $P$ can be modeled as the affine function $P(x) = Ex$, where $E$ is 32x32-matrix in $GF(2)$, and $x \in \mathbb{F}_2^{32}$. Therefore, the $i$-th round of the cipher can be rewrite as: $$\begin{aligned} r_i(x) &= P(S(x + K^i)) \\ &= P(C*(x + K^i) + D) \\ &= E * C * x + E * C * K^i + E * D \\ &= G * x + G * K^i + H \end{aligned}$$ where $G = E * C, H = E * D, K^i = (K_{32(i mod 4) + 1}, K_{32(i mod 4) + 2},..., K_{32(i mod 4) + 32})$

And the ciphertext can be represented as: $$\begin{aligned} c = Enc(m) &= K^{100} + r_{99}(r_{98}(...r_1(m))) \\ &= G^{99} * m + H * \frac{G^{99} - I}{G - I} + \sum_{i=0}^{99}G^iK^{100-i}\end{aligned}$$

With 100 random pairs of plaintext and ciphertext, Carol can calculate $U = H * \frac{G^{99} - I}{G - I} + \sum_{i=0}^{99}G^iK^{100-i}$ and $V = G^{99}$. Therefore, given random ciphertext $c_A = Enc(m_A)$ from Alice, Carol can recover plaintext by calculate $m_A = (c_A - U)/V$. QED